In [54]:
import pandas as pd
import os 
file = "/data/steam_games.csv"
file = os.getcwd() + file
raw_steam_data = pd.read_csv(file)

In [55]:
raw_steam_data['name'].size #40833 games
names = raw_steam_data['name'] 
names.size #40833 names (possibly not unique) 
types = raw_steam_data['types']
#types[types.duplicated(keep=False)].size

#example of duplicate names -> unsuitable for unique id
for i in range(0, raw_steam_data['name'].size):
    if "The Deer" == raw_steam_data['name'][i]:
        print(raw_steam_data.iloc[i])
        print(i)

url                         https://store.steampowered.com/app/400240/The_...
types                                                                     app
name                                                                 The Deer
desc_snippet                "The Deer" is an educational inter­ac­tive exp...
recent_reviews                                                            NaN
all_reviews                 Mostly Positive,(317),- 70% of the 317 user re...
release_date                                                      Dec 2, 2015
developer                                        Fabio Ferrara,Giulia Airoldi
publisher                                           Chubby Pixel,Chubby Pixel
popular_tags                Indie,Adventure,Education,Family Friendly,Shor...
game_details                Single-player,Full controller support,Steam Tr...
languages                                                             English
achievements                                                    

In [56]:
#purging unwanted data
purged_steam_data = raw_steam_data.drop(columns=['types', 'desc_snippet', 'achievements','game_description', 'minimum_requirements', 'recommended_requirements'])
purged_steam_data.iloc[5]

url               https://store.steampowered.com/bundle/5699/Gra...
name                     Grand Theft Auto V: Premium Online Edition
recent_reviews                                                  NaN
all_reviews                                                     NaN
release_date                                                    NaN
developer                                            Rockstar North
publisher                                            Rockstar Games
popular_tags                                                    NaN
game_details      Single-player,Multi-player,Downloadable Conten...
languages         English, French, Italian, German, Spanish - Sp...
genre                                              Action,Adventure
mature_content                                                  NaN
original_price                                                  NaN
discount_price                                               $35.18
Name: 5, dtype: object

In [57]:
import re
import math
amount_p = re.compile(r'\d+(,\d+)*')
value_p = re.compile(r'\d+')

def process_review(review):
    #print(review)
    if isinstance(review, str) :
        str_num = amount_p.search(review).group()
        nums = value_p.findall(str_num)
        amt = 0
        if 1 < len(nums):
            for i in range(0, len(nums) - 1):
                prev = nums[i + 1]
                power = len(prev)
                prev = int(prev)
                amt += int(nums[i])*(10**power) + prev
                
        else:
            amt = nums[0]
        
        all_nums = value_p.findall(review)
        if 2 < len(all_nums):
            val = all_nums[len(nums)]
        else:
            val = -1
            
        return [amt, val]
    else: 
        return []
    
def get_element(nums, index):
    if index < len(nums):
        return int(nums[index])
    else:
        return 0

recent_review_numbers = purged_steam_data.apply(lambda x: process_review(x['recent_reviews']), axis=1)
recent_review_numbers


purged_steam_data['amt_rec_revs'] = recent_review_numbers.apply(lambda x: get_element(x, 0))
purged_steam_data['rec_rev_score'] = recent_review_numbers.apply(lambda x: get_element(x, 1))

alltime_review_numbers = purged_steam_data.apply(lambda x: process_review(x['all_reviews']), axis=1)

purged_steam_data['amt_all_revs'] = alltime_review_numbers.apply(lambda x: get_element(x, 0))
purged_steam_data['all_rev_score'] = alltime_review_numbers.apply(lambda x: get_element(x, 1))

purged_steam_data = purged_steam_data.drop(columns=['recent_reviews','all_reviews'])

#keys for new columns:
#'amt_rec_revs' -> the amount of recent reviews
#'rec_rev_score' -> the percentage of the recent reviews
#'amt_all_revs' -> the amount of all reviews
#'all_rev_score' - > the perceent of all reviews


In [58]:
purged_steam_data

,url,name,release_date,developer,publisher,popular_tags,game_details,languages,genre,mature_content,original_price,discount_price,amt_rec_revs,rec_rev_score,amt_all_revs,all_rev_score
0,https://store.steampowered.com/app/379720/DOOM/,DOOM,"May 12, 2016",id Software,"Bethesda Softworks,Bethesda Softworks","FPS,Gore,Action,Demons,Shooter,First-Person,Gr...","Single-player,Multi-player,Co-op,Steam Achieve...","English,French,Italian,German,Spanish - Spain,...",Action,NaN,$19.99,$14.99,554,89,42550,92
1,https://store.steampowered.com/app/578080/PLAY...,PLAYERUNKNOWN'S BATTLEGROUNDS,"Dec 21, 2017",PUBG Corporation,"PUBG Corporation,PUBG Corporation","Survival,Shooter,Multiplayer,Battle Royale,PvP...","Multi-player,Online Multi-Player,Stats","English,Korean,Simplified Chinese,French,Germa...","Action,Adventure,Massively Multiplayer",Mature Content Description The developers de...,$29.99,NaN,6214,49,836608,49
2,https://store.steampowered.com/app/637090/BATT...,BATTLETECH,"Apr 24, 2018",Harebrained Schemes,"Paradox Interactive,Paradox Interactive","Mechs,Strategy,Turn-Based,Turn-Based Tactics,S...","Single-player,Multi-player,Online Multi-Player...","English,French,German,Russian","Action,Adventure,Strategy",NaN,$39.99,NaN,166,54,7030,71
3,https://store.steampowered.com/app/221100/DayZ/,DayZ,"Dec 13, 2018",Bohemia Interactive,"Bohemia Interactive,Bohemia Interactive","Survival,Zombies,Open World,Multiplayer,PvP,Ma...","Multi-player,Online Multi-Player,Steam Worksho...","English,French,Italian,German,Spanish - Spain,...","Action,Adventure,Massively Multiplayer",NaN,$44.99,NaN,932,57,167115,61
4,https://store.steampowered.com/app/8500/EVE_On...,EVE Online,"May 6, 2003",CCP,"CCP,CCP","Space,Massively Multiplayer,Sci-fi,Sandbox,MMO...","Multi-player,Online Multi-Player,MMO,Co-op,Onl...","English,German,Russian,French","Action,Free to Play,Massively Multiplayer,RPG,...",NaN,Free,NaN,287,54,11481,74
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40828,https://store.steampowered.com/app/899836/Rock...,Rocksmith® 2014 Edition – Remastered – Sabaton...,"Feb 12, 2019",Ubisoft - San Francisco,NaN,"Casual,Simulation","Single-player,Shared/Split Screen,Downloadable...","English,German,French,Italian,Spanish - Spain,...","Casual,Simulation",NaN,$2.99,NaN,0,0,0,0
40829,https://store.steampowered.com/app/899832/Rock...,Rocksmith® 2014 Edition – Remastered – Stone T...,"Feb 5, 2019",Ubisoft - San Francisco,NaN,"Casual,Simulation","Single-player,Shared/Split Screen,Downloadable...","English,German,French,Italian,Spanish - Spain,...","Casual,Simulation",NaN,$2.99,NaN,0,0,0,0
40830,https://store.steampowered.com/app/906840/Fant...,Fantasy Grounds - Quests of Doom 4: A Midnight...,"Jul 31, 2018","SmiteWorks USA, LLC",NaN,"RPG,Indie,Strategy,Software,Turn-Based,Fantasy...","Multi-player,Co-op,Cross-Platform Multiplayer,...",English,"Indie,RPG,Strategy",NaN,$7.99,NaN,0,0,0,0
40831,https://store.steampowered.com/app/906635/Mega...,Mega Man X5 Sound Collection,"Jul 24, 2018","CAPCOM CO., LTD","CAPCOM CO., LTD,CAPCOM CO., LTD",Action,"Single-player,Downloadable Content,Steam Achie...","English,French,Italian,German,Spanish - Spain,...",Action,NaN,$9.99,NaN,0,0,0,0
